In [87]:
import spinetools
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from copy import deepcopy
import random
from plotly.subplots import make_subplots

np.set_printoptions(precision=2)


root_directory = None
view_0 = 'PA0'
view_1 = 'LAT'

# Preamble
Please write the RMS calculation function here. You will need this function throughout the assignment, so craft it with care !

In [88]:
import numpy as np

def rms(reference: np.ndarray, prediction: np.ndarray) -> float:
    """
    Calculates RMS

    Parameters
    ----------
    reference : np.ndarray
        Reference value
    prediction : np.ndarray
        Calculated/predicted value

    Returns
    -------
    float
        RMS metric
    """
    if reference.shape != prediction.shape:
        raise ValueError('Please provide arrays that are shaped the same! Reference has shape {refshape} whereas prediction has shape {predshape}'.format(refshape=reference.shape, predshape=prediction.shape))

    # For 2D arrays (N, 3) where we calculate RMS for each axis (X, Y, Z)
    if len(reference.shape) == 2 and reference.shape[1] == 3:
        rms_value = np.sqrt(np.mean(np.sum((reference - prediction) ** 2, axis=1)))
    
    # For 1D arrays (N, ) where we calculate a single RMS value
    elif len(reference.shape) == 1:
        rms_value = np.sqrt(np.mean((reference - prediction) ** 2))
    
    else:
        raise ValueError('Invalid array shape. Expected (N,) or (N, 3), got {}'.format(reference.shape))

    return rms_value



In [89]:
# 1D
reference_1d = np.array([1, 2, 3, 4, 5])
prediction_1d = np.array([2, 2, 3, 4, 6])

# Manually calculating the RMS for this simple case:
# Difference = [1, 0, 0, 0, 1]
# Squared difference = [1, 0, 0, 0, 1]
# Mean of squared differences = (1+0+0+0+1)/5 = 0.4
# RMS = sqrt(0.4) = 0.6324555320336759

print("1D RMS:", rms(reference_1d, prediction_1d))  # Expected: 0.6324555320336759
# 2D Test Case: 2D arrays where each row is (x, y, z) coordinate
reference_2d = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
prediction_2d = np.array([[1, 2, 3], [4, 4, 6], [8, 8, 9]])

# Manually calculating RMS for this case:
# For each point:
# Point 1 difference: [0, 0, 0]
# Point 2 difference: [0, 1, 0]
# Point 3 difference: [1, 0, 0]
# Squared differences: [0, 0, 0], [0, 1, 0], [1, 0, 0]
# Sum of squared differences for each point: [0, 1, 1]
# Mean of the sums: (0 + 1 + 1) / 3 = 0.6667
# RMS = sqrt(0.6667) = 0.816496580927726

print("2D RMS:", rms(reference_2d, prediction_2d))  # Expected: 0.816496580927726



1D RMS: 0.6324555320336759
2D RMS: 0.816496580927726


# Question 1.1
Write the code for finding the over determined system matrix A. Then use the dlt function from spinetools.solver to extract the L parameters vector for each view.

In [90]:
root_directory = "Datafiles_LAB1/"
beads_data = spinetools.io.process_files(root_directory)
# ? Filter out non redundant beads
unique_beads = beads_data['2d']['bead'].tolist()
unique_beads = [x.replace('C', 'A') for x in unique_beads]
unique_beads = [x.replace('D', 'B') for x in unique_beads]

# ? Count values
value_count = {}
for k in unique_beads:
    if k in value_count.keys():
        value_count[k] += 1
    else:
        value_count[k] = 1

# ? Exclude the ones that aren't visible from everywhere
for k, v in zip(list(value_count.keys()), list(value_count.values())):
    if v == 1:
        print('Dropped', k)
        del value_count[k]

unique_beads = list(value_count.keys())
unique_beads.extend([x.replace('A', 'C') if 'A' in x else x.replace('B', 'D') for x in unique_beads])
beads_data['2d'] = beads_data['2d'][beads_data['2d']['bead'].isin(unique_beads)]
beads_data['3d'] = beads_data['3d'][beads_data['3d']['bead'].isin(unique_beads)]

Dropped A_4_1
Dropped B_3_5


In [91]:
from spinetools.data import select_view, join_dataframes

def get_a(view:str, data:dict) -> np.ndarray:
    """
    Uses the view name and existing data for pre calculating the system matrix for calculating the 11 DLT coefficients

    Parameters
    ----------
    view : str
        View name
    data : dict
        Data as a dict, with two keys ['2d', '3d']. Each key is associated to a table giving points location in 2d/3d, bead identifier and view name.

    Returns
    -------
    np.ndarray
        A matrix, for the DLT
    """
    A = []
    joint_data = join_dataframes(select_view(data['2d'], view), data['3d'], 'bead')
    print(joint_data.columns)

    # TODO : Using the equations from Appendix 1, write a function to turn pairs of 2D/3D beads data to an overdeterminated system.
    # HINT 1 : we want to end up with some system AM = 0 where M is a vector containing [L0, L1... L10, 1]. So you just need to return A !
    # HINT 2 : I gave you 'joint_data'. It's a dataframe, so check the columns names !
    for i, row in joint_data.iterrows():
        #excract coordinates
        X,Y,Z=row["x_3d"], row["y_3d"], row["z_3d"]
        u,v=row['x_2d'],row['y_2d']
        # !! don't forget constant term to have 11 parameters
        row_u = [
            X, Y, Z, 1, 0, 0, 0, 0, -u * X, -u * Y, -u * Z,-u
        ]
        row_v = [
            0, 0, 0, 0, X, Y, Z, 1, -v * X, -v * Y, -v * Z,-v
        ]
        
        A.append(row_u)
        A.append(row_v)
    return np.array(A)

In [92]:
l_view0 = spinetools.solver.dlt(get_a('Beads2D_'+view_0, beads_data))
l_view1 = spinetools.solver.dlt(get_a('Beads2D_'+view_1, beads_data))

Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')


In [93]:
print("DLT parameters for %s : "%view_0, l_view0)
print("DLT parameters for %s : "%view_1, l_view1)

DLT parameters for PA0 :  [ 4.09e-03 -2.57e+00 -2.44e-03 -4.16e-03 -3.60e-02  3.18e-03  2.58e+00
 -5.67e-01  5.50e-04  1.57e-06 -9.44e-06]
DLT parameters for LAT :  [-3.09e+00  1.29e-01 -1.93e-03 -9.36e+02  4.54e-03  4.34e-02  3.10e+00
  1.39e+01 -2.36e-05 -6.61e-04 -1.22e-05]


In [94]:
#Test 

# Question 1.2

In [95]:
def get_parameters(dlt_params:np.ndarray) -> dict:
    """
    Outputs a dictionnary containing the intrinsic and extrinsic parameters of the system based on the DLT parameters.

    Parameters
    ----------
    dlt_params : np.ndarray
        _description_

    Returns
    -------
    dict
        Dict with keys ['u0', 'v0', 'c_u', 'c_v', 'R', 'source_coordinates']
    """
    out = {}
    L1, L2, L3, L4, L5, L6, L7, L8, L9, L10, L11 = dlt_params
    #
    # TODO : Using the equations from Appendix 2, write a function to calculate intrinsic and extrinsic parameters of a system based on the DLT parameters
    #----------equations of appendix B----------------#
    d = -1/np.sqrt(L9**2 + L10**2 + L11**2) #scaling factor
    #intrinsic params
    u0 = (L1* L9 + L2 * L10 + L3 * L11) * d**2
    v0 = (L5 * L9 + L6 * L10 + L7 * L11) * d**2
    #focal lengths
    c_u = np.sqrt(d**2 * ((u0 * L9 - L1)**2 + (u0 * L10 - L2)**2 + (u0 * L11 - L3)**2))
    c_v = np.sqrt(d**2 * ((v0 * L9 - L5)**2 + (v0 * L10 - L6)**2 + (v0 * L11 - L7)**2))
    #Rotation matrix
    R = np.array([
        [d / c_u * (u0 * L9 - L1), d / c_u * (u0 * L10 - L2), d / c_u * (u0 * L11 - L3)],
        [d / c_v * (v0 * L9 - L5), d / c_v * (v0 * L10 - L6), d / c_v * (v0 * L11 - L7)],
        [L9 * d, L10 * d, L11 * d]
    ])
    #source coordinates -R^-1 * T, with T= [L_4,L_8,1]^t the translation vector
    source_coordinates = np.linalg.inv(R).dot([-L4, -L8, -1])
    out = {'u0': u0,'v0': v0,'c_u': c_u,'c_v': c_v,'R': R,'source_coordinates': source_coordinates}
    return out

print('View %s :'%view_0)
for k, v in get_parameters(l_view0).items():
    print('\t {variable} = {value}'.format(variable = k, value = v))
print('-------------------------------------------------------------')
print('View %s :'%view_1)
for k, v in get_parameters(l_view1).items():
    print('\t {variable} = {value}'.format(variable = k, value = v))

View PA0 :
	 u0 = -5.818320392632722
	 v0 = -145.91496533853473
	 c_u = 4667.8104535235125
	 c_v = 4684.955368831336
	 R = [[ 2.84e-03 -1.00e+00 -9.71e-04]
 [ 1.72e-02  1.32e-03  1.00e+00]
 [-1.00e+00 -2.85e-03  1.72e-02]]
	 source_coordinates = [ 1.01 -0.    0.55]
-------------------------------------------------------------
View LAT :
	 u0 = -29.21576695254617
	 v0 = -152.70327971664815
	 c_u = 4673.68335022721
	 c_v = 4690.201491353493
	 R = [[-9.99e-01  3.56e-02 -7.39e-04]
 [ 3.04e-04 -1.85e-02  1.00e+00]
 [ 3.56e-02  9.99e-01  1.85e-02]]
	 source_coordinates = [-935.29   32.57  -13.04]


# Question 1.3

In [96]:
def calculate_3d_point(u0:float, v0:float, u1:float, v1:float, L0:np.ndarray, L1:np.ndarray) -> np.ndarray:
    """
    Calculates a 3D point from 2 2D coordinates pairs and L0 & L1 DLT calibration coefficients

    Parameters
    ----------
    u0 : float
        point 1 x-coordinate
    v0 : float
        point 1 y-coordinate
    u1 : float
        point 2 x-coordinate
    v1 : float
        point 2 y-coordinate
    L0 : np.ndarray
        DLT coefficients for view 1
    L1 : np.ndarray
        DLT coefficients for view 2
    """
    A = []
    # TODO : using Appendix A, write a function that takes 2 pairs of 2D coordinates and 2 11-parameters vectors as an input to return the corresponding 3D point. 
    # HINT 1 : It's the exact same problem as in question 1.1, but now the unknowns are x,y and z !
    # HINT 2 : Refactor the pair of equation in order to have a 4 equations system (2 equations per view, 2 views) for finding a triplet of coordinates : AM = 0 where M = [x, y, z, 1]
    
    #return spinetools.solver.dlt(np.array(A))
    # to solve we need at least 2 sets of projection equations: 
    
    A = [
        # PA0 view - u0 equation
        [u0 * L0[8] - L0[0], u0 * L0[9] - L0[1], u0 * L0[10] - L0[2], u0 - L0[3]],
        # PA0 view - v0 equation
        [v0 * L0[8] - L0[4], v0 * L0[9] - L0[5], v0 * L0[10] - L0[6], v0 - L0[7]],
        # LAT view - u1 equation
        [u1 * L1[8] - L1[0], u1 * L1[9] - L1[1], u1 * L1[10] - L1[2], u1 - L1[3]],
        # LAT view - v1 equation
        [v1 * L1[8] - L1[4], v1 * L1[9] - L1[5], v1 * L1[10] - L1[6], v1 - L1[7]]
    ]
    
    # Solve the system using DLT 
    # M will give us [X, Y, Z, 1] -> We are interested in X, Y, Z
    M = spinetools.solver.dlt(np.array(A))
    
    return M[:3]  #x,y,z

In [97]:

pio.renderers.default = 'browser'  # Change this if running outside Jupyter

# Load the spine structure and the beads data
spine = spinetools.structures.Spine(os.path.join(root_directory, 'Vertebrae2D.mat'))
colors = px.colors.qualitative.Alphabet
fig = spinetools.render.Plot3D()

for i, v_name in enumerate(spine.vertebrae):
    # TODO : for each vertebra, calculate the location of each anatomical landmark and add its x,y,z coordinates to vert_x, vert_y, vert_z. Each vertebra will then be added to the 3D plot and plotted in the end.
    vert = spine.get(v_name).data # Returns the vertebra data
    """
    HINT : the data is a table containing the landmark name and the x,y coordinates for each view (so 5 columns)
    HINT : Print the variable vert, and add a 'break' instruction to get out of the loop
    For each vertebra, you should have 6 points. Each point is linked to 2 pairs of 2D coordinates, one in each view. These coordinates, with the two 11-DLT parameters will allow you to calculate the 3D position of the 6 anatomical landmarks on the vertebra.
    """
    vert_x, vert_y, vert_z = [], [], []
    """
    PSEUDOCODE
    for each anatomical_landmark in vertebra_points:
        x_point, y_point, z_point <- calculate_3d_point(anatomical_landmark, l_view0, l_view1)
        Append x_point to vert_x
        Append y_point to vert_y
        Append z_point to vert_z
    """
    # for each anatomical_landmark in vertebra_points
    for j, row in vert.iterrows():
        # 2D PA0
        u0, v0 = row['x_Vertebrae_PA0'], row['y_Vertebrae_PA0']
        
        # 2D LAT
        u1, v1 = row['x_Vertebrae_LAT'], row['y_Vertebrae_LAT']
        
        # 3D using DLT 
        x, y, z = calculate_3d_point(u0, v0, u1, v1, l_view0, l_view1)
        
        vert_x.append(x)
        vert_y.append(y)
        vert_z.append(z)
    
    
    fig.scatter_vertebrae(vert_x, vert_y, vert_z, v_name, colors[i]) # Adds the landmarks to the figure
SPINE_3D_POINTS = fig.points # Access all the figure points as an array this way, will be useful in the rest of this assignment
fig.show() # Shows the figure

# Question 2.1

Here I recommend you write a function that samples `n_beads` in the full set, and returns the subset. A code skeleton and some hints can be found below.

In [98]:
def sample_n_beads(original_beads:dict, n_beads:int) -> dict:
    """
    Samples n_beads beads from the original beads set

    Parameters
    ----------
    original_beads : dict
        Dictionnary containing information on 2D and 3D position of the whole set of calibration beads
    n_beads : int
        Number of beads to select

    Returns
    -------
    dict
        Dictionnary containing information on 2D and 3D position of the subsampled of calibration beads
    """
    output = deepcopy(original_beads) # ? First we deepcopy the original beads data, to avoid modifying it by mistake
    selected_beads = []
    """
    Let's modify our output until we reach the desired number of beads.
    You want n beads sampled across both plates A & B (remember that plates C & D are just A & B rotated):
    1. So you're gonna sample n_beads // 2 beads on plate A, and n_beads - (n_beads // 2) beads on plate B.
    2. Add all these beads' names in the selected_beads list.
    3. Then for each A & B bead, you want to add it's counterpart of plate C & D.
    The selection step is already done :)
    """
    all_beads = output['2d']['bead'].tolist()
    plateA_beads = [bead for bead in all_beads if bead.startswith('A')]
    plateB_beads = [bead for bead in all_beads if bead.startswith('B')]
    selectedA_beads = random.sample(plateA_beads, n_beads//2)
    selectedB_beads = random.sample(plateB_beads, n_beads - n_beads//2)
    
    corresponding_C_beads = [bead.replace('A', 'C', 1) for bead in selectedA_beads]
    corresponding_D_beads = [bead.replace('B', 'D', 1) for bead in selectedB_beads]

    selected_beads = selectedA_beads + selectedB_beads + corresponding_C_beads + corresponding_D_beads

    output['2d'] = output['2d'][output['2d']['bead'].isin(selected_beads)].sort_values(by = 'bead', ignore_index = True)
    output['3d'] = output['3d'][output['3d']['bead'].isin(selected_beads)].sort_values(by = 'bead', ignore_index = True)
    return output

In [99]:
# Let's increase the number of calibration beads gradually
spine = spinetools.structures.Spine(os.path.join(root_directory,'Vertebrae2D.mat')) # Initialize the vertebrae structure

# Create a dictionnary to store the results. Access a dictionnary key with brackets, for instance rms_measurements['x']
rms_measurements = {
        'x' : [],
        'y' : [],
        'z' : [],
        '3d' : [],
        'n_beads':[]
    }

"""
PSEUDOCODE
computed_points_3d <- list
for n_beads in [1...N], do:
    beads_subset <- sample_n_beads(beads_data, n_beads)
    A_view_0 <- get_a(view_0, beads_subset)
    l_view_0 <- dlt(A_view_0)
    A_view_1 <- get_a(view_1, beads_subset)
    l_view_1 <- dlt(A_view_1)
    for each vertebra in vertebrae:
        for each anatomical_landmark in vertebra_points:
        x_point, y_point, z_point <- calculate_3d_point(anatomical_landmark, l_view0, l_view1)
        Append (x_point, y_point, z_point) to computed_points_3d
    for each axis in ['x', ..., '3d']:
        Append rms(SPINE_3D_POINTS, computed_points_3d) to rms_measurements[axis]
        Append n_beads to rms_measurements['n_beads']
"""

max_beads = 22
min_beads = 5
for n_beads in range(max_beads, min_beads, -1):
    beads_subset = sample_n_beads(beads_data, n_beads)
    A_view_0 = get_a('Beads2D_'+view_0, beads_subset)
    l_view_0 = spinetools.solver.dlt(A_view_0)
    A_view_1 = get_a('Beads2D_'+view_1, beads_subset)
    l_view_1 = spinetools.solver.dlt(A_view_1)

    computed_points_3d = []

    for _, v_name in enumerate(spine.vertebrae):
        # TODO : for each vertebra, calculate the location of each anatomical landmark and add its x,y,z coordinates to vert_x, vert_y, vert_z. Each vertebra will then be added to the 3D plot and plotted in the end.
        vert = spine.get(v_name).data # Returns the vertebra data
         # for each anatomical_landmark in vertebra_points
        for j, row in vert.iterrows():
            # 2D PA0
            u0, v0 = row['x_Vertebrae_PA0'], row['y_Vertebrae_PA0']
            # 2D LAT
            u1, v1 = row['x_Vertebrae_LAT'], row['y_Vertebrae_LAT']
            x_point, y_point, z_point = calculate_3d_point(u0, v0, u1, v1, l_view_0, l_view_1)
            computed_points_3d.append([x_point, y_point, z_point])

    computed_points_3d = np.array(computed_points_3d)

    rms_measurements['x'].append(rms(SPINE_3D_POINTS[:,0], computed_points_3d[:,0]))
    rms_measurements['y'].append(rms(SPINE_3D_POINTS[:,1], computed_points_3d[:,1]))
    rms_measurements['z'].append(rms(SPINE_3D_POINTS[:,2], computed_points_3d[:,2]))
    rms_measurements['3d'].append(rms(SPINE_3D_POINTS, computed_points_3d))
    rms_measurements['n_beads'].append(n_beads)

Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='o

# Plotly interlude

In [100]:
# Convert the number of beads and RMS values into numpy arrays for easier plotting
n_beads = np.array(rms_measurements['n_beads'])
rms_x = np.array(rms_measurements['x'])
rms_y = np.array(rms_measurements['y'])
rms_z = np.array(rms_measurements['z'])
rms_3d = np.array(rms_measurements['3d'])

# Create the figure
fig = go.Figure()

# Add traces for each RMS measurement
fig.add_trace(go.Scatter(
    x=n_beads,
    y=rms_x,
    mode='lines+markers',
    name='RMS X',
    line=dict(shape='spline'),
))

fig.add_trace(go.Scatter(
    x=n_beads,
    y=rms_y,
    mode='lines+markers',
    name='RMS Y',
    line=dict(shape='spline'),
))

fig.add_trace(go.Scatter(
    x=n_beads,
    y=rms_z,
    mode='lines+markers',
    name='RMS Z',
    line=dict(shape='spline'),
))

fig.add_trace(go.Scatter(
    x=n_beads,
    y=rms_3d,
    mode='lines+markers',
    name='RMS 3D',
    line=dict(shape='spline'),
))

# Update the layout
fig.update_layout(
    title='RMS Values vs. Number of Beads',
    xaxis_title='Number of Beads',
    yaxis_title='RMS Value',
    legend=dict(title='RMS Metrics'),
)

# Show the figure
fig.show()

In [101]:
# TODO : On the same graph, plot all your RMS values wrt. the number of used calibration beads !

# Question 2.2

Here are 4 sets of 8 calibration beads. For each set, determine which one is which (same plate, small volume, medium volume, large volume) and plot calibration error curves.  
Don't hesitate to use the function `plot_selected_beads` from `spinetools.render`to vizualize the selected beads to sort out the subsets !

In [102]:
set_0 = ['A_1_6', 'A_4_6', 'A_1_2', 'A_4_2', 'B_1_5', 'B_5_5', 'B_1_1', 'B_5_1']
set_1 = ['B_1_1', 'B_2_2', 'B_4_2', 'B_5_3', 'B_1_4', 'B_2_5', 'B_4_3', 'B_5_5']
set_2 = ['A_1_4', 'A_2_4', 'A_1_5', 'A_2_5', 'B_1_3', 'B_2_3', 'B_1_4', 'B_2_4']
set_3 = ['A_1_4', 'A_3_4', 'A_1_5', 'A_3_5', 'B_1_3', 'B_3_3', 'B_1_4', 'B_3_4']

spinetools.render.plot_selected_beads(beads_data, set_0) # Large volume
spinetools.render.plot_selected_beads(beads_data, set_1) # Same plate
spinetools.render.plot_selected_beads(beads_data, set_2) # Small volume
spinetools.render.plot_selected_beads(beads_data, set_3) # Medium volume

large_volume, same_plate, small_volume, medium_volume = set_0, set_1, set_2, set_3

TODO : same thing as before, but now you have to select the provided beads ! Look at the function `sample_n_beads` from Q2.1 if you need some help on how to do this.

In [103]:
# Re-order sets from small volume to large volume + same plate
sets = [small_volume, medium_volume, large_volume, same_plate]
set_names = ['Small volume', 'Medium volume', 'Large volume', 'Same plate']

for i, set_ in enumerate(sets): 
    # Add counterparts to subsets
    sets[i] += [bead.replace('A', 'C').replace('B', 'D') for bead in set_]

    beads_set = deepcopy(beads_data)
    
    beads_set['2d'] = beads_set['2d'][beads_set['2d']['bead'].isin(sets[i])].sort_values(by='bead', ignore_index=True)
    beads_set['3d'] = beads_set['3d'][beads_set['3d']['bead'].isin(sets[i])].sort_values(by='bead', ignore_index=True)
    
    # Assign the modified beads_set back to the original list
    sets[i] = beads_set    

rms_measurements = {
        'x' : [],
        'y' : [],
        'z' : [],
        '3d' : [],
    }

for i, beads_subset in enumerate(sets):
    A_view_0 = get_a('Beads2D_'+view_0, beads_subset)
    l_view_0 = spinetools.solver.dlt(A_view_0)
    A_view_1 = get_a('Beads2D_'+view_1, beads_subset)
    l_view_1 = spinetools.solver.dlt(A_view_1)

    computed_points_3d = []

    for _, v_name in enumerate(spine.vertebrae):
        # TODO : for each vertebra, calculate the location of each anatomical landmark and add its x,y,z coordinates to vert_x, vert_y, vert_z. Each vertebra will then be added to the 3D plot and plotted in the end.
        vert = spine.get(v_name).data # Returns the vertebra data
         # for each anatomical_landmark in vertebra_points
        for j, row in vert.iterrows():
            # 2D PA0
            u0, v0 = row['x_Vertebrae_PA0'], row['y_Vertebrae_PA0']
            # 2D LAT
            u1, v1 = row['x_Vertebrae_LAT'], row['y_Vertebrae_LAT']
            x_point, y_point, z_point = calculate_3d_point(u0, v0, u1, v1, l_view_0, l_view_1)
            computed_points_3d.append([x_point, y_point, z_point])

    computed_points_3d = np.array(computed_points_3d)

    rms_measurements['x'].append(rms(SPINE_3D_POINTS[:,0], computed_points_3d[:,0]))
    rms_measurements['y'].append(rms(SPINE_3D_POINTS[:,1], computed_points_3d[:,1]))
    rms_measurements['z'].append(rms(SPINE_3D_POINTS[:,2], computed_points_3d[:,2]))
    rms_measurements['3d'].append(rms(SPINE_3D_POINTS, computed_points_3d))

# Plotting
fig = go.Figure()

for axis in ['x', 'y', 'z', '3d']:
    fig.add_trace(go.Bar(
        name=f'RMS {axis}',
        x=set_names[:3],  # Subset names
        y=rms_measurements[axis],  # RMS values for the current axis
        text=[f'{v:.4f}' for v in rms_measurements[axis]],  # Add text labels
        textposition='auto'
    ))

# Update the layout of the plot
fig.update_layout(
    title="Calibration Errors (RMS) for Each Subset",
    barmode='group',  # Group bars for each category
    xaxis_title="Subsets",
    yaxis_title="RMS Error",
    legend_title="Axes",
    template="plotly_white"
)

# Show the plot
fig.show()


# RMS for calibration beads on the same plate
print(f'Calibration beads on the same plate:')
for axis, values in rms_measurements.items():
    print(f'RMS {axis} : {values[3]}')

Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Calibration beads on the same plate:
RMS x : 349.3594665527344
RMS y : 51.47358703613281
RMS z : 106.58932495117188
RMS 3d : 368.86700439453125


# Question 2.3

Using the **small** calibration volume from previous question, plot reconstruction error with respect to the center of gravity of the calibration volume !

In [104]:
small_volume_subset = sets[0]

def calculate_COG(beads_set:dict):
    cog_x = beads_set['3d']['x_3d'].mean()
    cog_y = beads_set['3d']['y_3d'].mean()
    cog_z = beads_set['3d']['z_3d'].mean()
    return [cog_x, cog_y, cog_z]

def distance_between(p1, p2):
    return np.sqrt( (p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2 )

COG = calculate_COG(small_volume_subset)
print(f'Center Of Gravity small volume subset: {COG}')

errors = {
    'x' : [],
    'y' : [],
    'z' : [],
    '3d' : [],
    'distance_to_cog' : []
}

A_view_0 = get_a('Beads2D_'+view_0, small_volume_subset)
l_view_0 = spinetools.solver.dlt(A_view_0)
A_view_1 = get_a('Beads2D_'+view_1, small_volume_subset)
l_view_1 = spinetools.solver.dlt(A_view_1)

computed_points_3d = []

for i, v_name in enumerate(spine.vertebrae):
    # TODO : for each vertebra, calculate the location of each anatomical landmark and add its x,y,z coordinates to vert_x, vert_y, vert_z. Each vertebra will then be added to the 3D plot and plotted in the end.
    vert = spine.get(v_name).data # Returns the vertebra data
        # for each anatomical_landmark in vertebra_points
    for j, row in vert.iterrows():
        # 2D PA0
        u0, v0 = row['x_Vertebrae_PA0'], row['y_Vertebrae_PA0']
        # 2D LAT
        u1, v1 = row['x_Vertebrae_LAT'], row['y_Vertebrae_LAT']
        x_point, y_point, z_point = calculate_3d_point(u0, v0, u1, v1, l_view_0, l_view_1)
        computed_points_3d.append([x_point, y_point, z_point])

        errors['x'].append(abs(SPINE_3D_POINTS[i][0] - x_point))
        errors['y'].append(abs(SPINE_3D_POINTS[i][1] - y_point))
        errors['z'].append(abs(SPINE_3D_POINTS[i][2] - z_point))
        errors['3d'].append(distance_between(SPINE_3D_POINTS[i], [x_point, y_point, z_point]))
        errors['distance_to_cog'].append(distance_between(SPINE_3D_POINTS[i], COG))

computed_points_3d = np.array(computed_points_3d)

Center Of Gravity small volume subset: [-256.91019375, 45.61763124999999, -20.857125000000003]
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')


In [105]:
small_volume_subset = sets[0]

def calculate_COG(beads_set:dict):
    cog_x = beads_set['3d']['x_3d'].mean()
    cog_y = beads_set['3d']['y_3d'].mean()
    cog_z = beads_set['3d']['z_3d'].mean()
    return [cog_x, cog_y, cog_z]

def distance_between(p1, p2):
    return np.sqrt( (p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2 )

COG = calculate_COG(small_volume_subset)
print(f'Center Of Gravity small volume subset: {COG}')

errors = {
    'x' : [],
    'y' : [],
    'z' : [],
    '3d' : [],
    'distance_to_cog' : []
}

A_view_0 = get_a('Beads2D_'+view_0, small_volume_subset)
l_view_0 = spinetools.solver.dlt(A_view_0)
A_view_1 = get_a('Beads2D_'+view_1, small_volume_subset)
l_view_1 = spinetools.solver.dlt(A_view_1)

computed_points_3d = []

for i, v_name in enumerate(spine.vertebrae):
    # TODO : for each vertebra, calculate the location of each anatomical landmark and add its x,y,z coordinates to vert_x, vert_y, vert_z. Each vertebra will then be added to the 3D plot and plotted in the end.
    vert = spine.get(v_name).data # Returns the vertebra data
        # for each anatomical_landmark in vertebra_points
    for j, row in vert.iterrows():
        # 2D PA0
        u0, v0 = row['x_Vertebrae_PA0'], row['y_Vertebrae_PA0']
        # 2D LAT
        u1, v1 = row['x_Vertebrae_LAT'], row['y_Vertebrae_LAT']
        x_point, y_point, z_point = calculate_3d_point(u0, v0, u1, v1, l_view_0, l_view_1)
        computed_points_3d.append([x_point, y_point, z_point])

        point_number = i*6+j    # 6 points for each vertebra 

        errors['x'].append(abs(SPINE_3D_POINTS[i*6+j][0] - x_point))
        errors['y'].append(abs(SPINE_3D_POINTS[i*6+j][1] - y_point))
        errors['z'].append(abs(SPINE_3D_POINTS[i*6+j][2] - z_point))
        errors['3d'].append(distance_between(SPINE_3D_POINTS[i*6+j], [x_point, y_point, z_point]))
        errors['distance_to_cog'].append(distance_between(SPINE_3D_POINTS[i*6+j], COG))

computed_points_3d = np.array(computed_points_3d)

Center Of Gravity small volume subset: [-256.91019375, 45.61763124999999, -20.857125000000003]
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')
Index(['bead', 'x_2d', 'y_2d', 'x_3d', 'y_3d', 'z_3d'], dtype='object')


# Question 3.1 

Here you have to code the function that allow us to add noise to the calibration beads coordinates. Hint : you should take a look at the documentation of `np.random.normal` :)

In [108]:
def add_noise_2d(original_beads:dict, std:float) -> dict:
    """
    Adds noise to the 2D coordinates of the beads

    Parameters
    ----------
    original_beads : dict
        Dictionnary containing information on 2D and 3D position of the a set of calibration beads
    std : float
        Standard deviation for the noise added.

    Returns
    -------
    dict
        Noised beads set
    """
    output = deepcopy(original_beads) # ? First we deepcopy the original beads data, to avoid modifying it by mistake
    # TODO : select 2D beads and add noise to them
    for bead in output['2d']:
        bead['x_2d'] += np.random.normal(0, std)  # Add noise to x_2d coordinate
        bead['y_2d'] += np.random.normal(0, std)  # Add noise to y_2d coordinate
    return output

def add_noise_3d(original_beads:dict, std:float) -> dict:
    """
    Adds noise to the 3D coordinates of the beads

    Parameters
    ----------
    original_beads : dict
        Dictionnary containing information on 2D and 3D position of the a set of calibration beads
    std : float
        Standard deviation for the noise added.

    Returns
    -------
    dict
        Noised beads set
    """
    output = deepcopy(original_beads) # ? First we deepcopy the original beads data, to avoid modifying it by mistake
    # TODO : select 3D beads and add noise to them
    for bead in output['3d']:
        bead['x_3d'] += np.random.normal(0, std)  # Add noise to x_3d coordinate
        bead['y_3d'] += np.random.normal(0, std)  # Add noise to y_3d coordinate
        bead['z_3d'] += np.random.normal(0, std)  # Add noise to z_3d coordinate
    
    return output

# Question 3.2

# Question 4.1

# Question 4.2

This question is a bit tricky, as you have to get measurements for couples of values of standard deviation for 2D and 3D noise. A line plot is not very adapted for this, but we could use a heatmap ! Here is a small example :

In [107]:
fig = go.Figure()
heatmap = np.random.rand(11, 11)
fig.add_trace(
    go.Heatmap(
        z = heatmap,
        x = np.linspace(0., 10., 11),
        y = np.linspace(0., 10., 11),
        text = heatmap.round(2),
        texttemplate="%{text}",
        colorscale='turbo'
    ))
fig.update_xaxes(title_text='Axis 1')
fig.update_yaxes(title_text='Axis 2')
fig.update_layout(title = 'Plot title')
fig.update_traces(showscale=False)
fig.show()